# 0. Set Environment

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import torchtext
torchtext.disable_torchtext_deprecation_warning()
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint

import subprocess
import os
import sys

import datasets

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Selected device:", device)

Selected device: cuda


In [ ]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
model_dir = './models/seq2seq_Translation_model.pth'
pretrained_embedding_dir = './models/Glove_pretrained.pth'

# 1. Data processing

## 1-1. Get Data

## 1-2. Tokenize

## 1-3. Build Vocab 

## 1-4. Numericalize Text

## 1-5. Word Embedding

## 1-6. Prepare for Data Loading

# 2. Define Model

## 2-1. Model Structure

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, bias = True):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.LSTM = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, bias = bias)

    def forward(self, inputs):

        batch_size = inputs.size(0)

        x = self.embedding(inputs)

        h = torch.zeros(batch_size, self.hidden_dim)
        c = torch.zeros(batch_size, self.hidden_dim)

        output, (h, c) = self.LSTM(x, (h, c))
        
        return h, c

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, bias = True):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.LSTM = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, bias = bias)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, inputs, h, c):

        batch_size = inputs.size(0)
        seq_length = inputs.size(1)

        x = self.embedding(inputs)

        output, (h, c) = self.LSTM(x, (h, c))
        
        result = []
        for t in range(seq_length):
            result.append(F.sigmoid(self.fc(output[t]))) # output 클래스의 개수는 단어 집합의 수와 동일. 다중 클래스 분류 문제.

        return result

In [ ]:
class seq2seq(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, bias = True):

        self.Encoder = Encoder(input_dim, embedding_dim, hidden_dim, bias = bias)
        self.Decoder = Decoder(input_dim, embedding_dim, hidden_dim, output_dim, bias = bias)

    def forward(self, inputs):

        h, c = self.Encoder.forward(inputs)
        output = self.Decoder.forward(inputs, h, c)

        return output

## 2-2. Hyperparameter & functions